In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score
import itertools

In [2]:
image_size = (224, 224)  # Standard size for VGG19 input
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2
)


In [3]:
train_data = datagen.flow_from_directory(
    'CT-Scans',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 396 images belonging to 3 classes.


In [8]:

validation_data = datagen.flow_from_directory(
    'CT-Scans',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 96 images belonging to 3 classes.


In [4]:
# Step 2: Define and Compile Models
# Model 1: VGG-19 Pretrained Model
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg19_custom = Sequential([
    vgg19_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Cancer, COVID-19, Normal
])

vgg19_custom.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Model 2: Custom Model with Six Fully Connected Layers
custom_model = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Cancer, COVID-19, Normal
])

custom_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Step 3: Train Both Models
epochs = 5

# Train VGG19 model
history_vgg19 = vgg19_custom.fit(train_data, validation_data=validation_data, epochs=epochs)

# Train custom model with fully connected layers
history_custom = custom_model.fit(train_data, validation_data=validation_data, epochs=epochs)

Epoch 1/5


c:\Users\anshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/13 ━━━━━━━━━━━━━━━━━━━━ 17:01 85s/step - accuracy: 0.4062 - loss: 1.5606

KeyboardInterrupt: 

In [ ]:
# Step 4: Evaluate and Compare Results
# Plotting accuracy and loss
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history_vgg19.history['accuracy'], label='VGG-19 Accuracy')
plt.plot(history_vgg19.history['val_accuracy'], label='VGG-19 Val Accuracy')
plt.plot(history_custom.history['accuracy'], label='Custom Model Accuracy')
plt.plot(history_custom.history['val_accuracy'], label='Custom Model Val Accuracy')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history_vgg19.history['loss'], label='VGG-19 Loss')
plt.plot(history_vgg19.history['val_loss'], label='VGG-19 Val Loss')
plt.plot(history_custom.history['loss'], label='Custom Model Loss')
plt.plot(history_custom.history['val_loss'], label='Custom Model Val Loss')
plt.title('Model Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [ ]:
# Predictions and true labels
y_true = validation_data.classes
y_pred_vgg19 = np.argmax(vgg19_custom.predict(validation_data), axis=1)
y_pred_custom = np.argmax(custom_model.predict(validation_data), axis=1)

# Confusion Matrix for VGG-19 Model
cm_vgg19 = confusion_matrix(y_true, y_pred_vgg19)
plt.figure()
plot_confusion_matrix(cm_vgg19, classes=train_data.class_indices.keys(), title='VGG-19 Confusion Matrix')

# Confusion Matrix for Custom Model
cm_custom = confusion_matrix(y_true, y_pred_custom)
plt.figure()
plot_confusion_matrix(cm_custom, classes=train_data.class_indices.keys(), title='Custom Model Confusion Matrix')


In [ ]:
# Display final accuracy scores
vgg19_accuracy = accuracy_score(y_true, y_pred_vgg19)
custom_accuracy = accuracy_score(y_true, y_pred_custom)
print(f"VGG-19 Model Accuracy: {vgg19_accuracy:.4f}")
print(f"Custom Model Accuracy: {custom_accuracy:.4f}")